In [1]:
import numpy as np
import cv2

def draw_line(event, x, y, flags, param):
    global prv_x, prv_y, prv_KFx, prv_KFy, Pos_x, Pos_y

    if event== cv2.EVENT_MOUSEMOVE:
        Pos_x, Pos_y= x, y
        cv2.line(img, (prv_x, prv_y), (x,y),(255, 0, 0), 3)
        cv2.line(img, (int(prv_KFx), int(prv_KFy)), (int(KF_x[0]), int(KF_x[2])), (0, 0, 255), 3)
        prv_x, prv_y= x, y
        prv_KFx, prv_KFy= KF_x[0], KF_x[2]

def get_sensor_data():
    global v, dt, Pos_x, Pos_y

    v= np.random.normal(0, 5)

    measurement_x= Pos_x+ v
    measurement_y= Pos_y+ v

    return measurement_x, measurement_y

def measurement_update(est_x, est_P, measurement_z):
    global H, R
    temp= (H@ est_P@ H.T) + R

    if temp.size== 1:
        K = est_P@ H.T / temp
    else:
        K = est_P@ H.T @ np.linalg.inv(temp)

    x= est_x+ K @ (measurement_z- H@ est_x)
    P= est_P- K @ H@ est_P
    return x, P

def state_prediction(prev_x, prev_P):
    global A, Q
    est_x= A@prev_x
    est_P= A@prev_P@A.T+ Q
    return est_x, est_P

# mainroutine
prv_x, prv_y, prv_KFx, prv_KFy= 0, 0, 0, 0
window_width= 1280
window_height= 760
dt= 0.03
# KalmanFilterSystem Model
A= np.array([[1,dt,0,0],[0,1,0,0],[0,0,1,dt],[0,0,0,1]],np.float_)
H= np.array([[1, 0, 0, 0], [0, 0, 1, 0]], np.float_)
Q= 1.0*np.eye(4)
R= np.array([[50, 0], [0, 50]], np.float_)
v= 0
KF_x= np.array([[0], [0], [0], [0]], np.float_)
P= 100*np.eye(4)
z= np.array([[0], [0]], np.float_)
Pos_x, Pos_y=0, 0

img= np.zeros((window_height, window_width,3), np.uint8)
cv2.namedWindow("CursorTracking", cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow("CursorTracking", window_width, window_height)
cv2.setMouseCallback('CursorTracking', draw_line)

while(1):
    cv2.imshow('CursorTracking', img)

    z[0], z[1] = get_sensor_data()
    est_x, est_P= state_prediction(KF_x, P)
    KF_x, P= measurement_update(est_x, est_P, z)

    if cv2.waitKey(int(dt*1000)) & 0xFF == 27:
        break
cv2.destroyWindow("CursorTracking")